In [81]:
# Install packages for project
# This has been commented as these packages have already been installed.

# %pip install pandas
# %pip install deep-translator
# %pip install google_trans_new
# %pip install google-cloud-translate==2.0.1
# %pip install trrex
# %pip install vaderSentiment


#Importing libraries

import glob
import pandas as pd
import numpy as np
from pandas import read_csv
from deep_translator import GoogleTranslator
from nltk import word_tokenize
import re
import eli5 as e5
import nltk
import csv
from nltk.corpus import stopwords
import pandas as pd
import trrex as tx
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
from sklearn.decomposition import LatentDirichletAllocation as LDA

nltk.download('vader_lexicon')
nltk.download('stopwords')

ModuleNotFoundError: No module named 'seaborn'

In [64]:
# read csv file

csv = pd.read_csv ('./data/McDonalds-Yelp-Sentiment-DFE.csv', encoding="latin-1", on_bad_lines='skip', delimiter = ',')
colums = csv.columns
print(colums)

csv.head(5)

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'policies_violated',
       'policies_violated:confidence', 'city', 'policies_violated_gold',
       'review'],
      dtype='object')


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,policies_violated,policies_violated:confidence,city,policies_violated_gold,review
0,679455653,False,finalized,3,2/21/15 0:36,RudeService\rOrderProblem\rFilthy,1.0\r0.6667\r0.6667,Atlanta,NaN,"I'm not a huge mcds lover, but I've been to be..."
1,679455654,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,Terrible customer service. Î¾I came in at 9:30...
2,679455655,False,finalized,3,2/21/15 0:26,SlowService\rOrderProblem,1.0\r1.0,Atlanta,NaN,"First they ""lost"" my order, actually they gave..."
3,679455656,False,finalized,3,2/21/15 0:27,na,0.6667,Atlanta,NaN,I see I'm not the only one giving 1 star. Only...
4,679455657,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,"Well, it's McDonald's, so you know what the fo..."


In [65]:
# Remove special characters and stopwords

stop = stopwords.words('english')
csv["review"] = csv["review"].apply(lambda x: " ".join(x.lower() for x in str(x).split() if x not in stop))

csv["review"] = csv["review"].map(lambda x: re.sub(r'[^A-Za-z0-9 ]+', '', x))
csv.head(5)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,policies_violated,policies_violated:confidence,city,policies_violated_gold,review
0,679455653,False,finalized,3,2/21/15 0:36,RudeService\rOrderProblem\rFilthy,1.0\r0.6667\r0.6667,Atlanta,NaN,im huge mcds lover ive better ones this far wo...
1,679455654,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,terrible customer service i came 930pm stood f...
2,679455655,False,finalized,3,2/21/15 0:26,SlowService\rOrderProblem,1.0\r1.0,Atlanta,NaN,first lost order actually gave someone one els...
3,679455656,False,finalized,3,2/21/15 0:27,na,0.6667,Atlanta,NaN,i see im one giving 1 star only 25 star thats ...
4,679455657,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,well mcdonalds know food is this review reflec...


In [66]:
# Tokenize the reviews and add them as a seperate column
csv['tokenized-review'] = csv['review'].apply(lambda x : nltk.word_tokenize(x))
csv['policies_violated'] = csv['policies_violated'].apply(lambda x : re.split('\r', str(x)))
csv['policies_violated:confidence'] = csv['policies_violated:confidence'].apply(lambda x : [float(x.strip(' []')) for x in re.split('\r', str(x))])

csv['tokenized-review'].head(5)

0    [im, huge, mcds, lover, ive, better, ones, thi...
1    [terrible, customer, service, i, came, 930pm, ...
2    [first, lost, order, actually, gave, someone, ...
3    [i, see, im, one, giving, 1, star, only, 25, s...
4    [well, mcdonalds, know, food, is, this, review...
Name: tokenized-review, dtype: object

In [67]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
csv['compound'] = [analyzer.polarity_scores(x)['compound'] for x in         
csv["review"]]
csv['neg'] = [analyzer.polarity_scores(x)['neg'] for x in         
csv["review"]]
csv['neu'] = [analyzer.polarity_scores(x)['neu'] for x in         
csv["review"]]
csv['pos'] = [analyzer.polarity_scores(x)['pos'] for x in         
csv["review"]]

csv

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,policies_violated,policies_violated:confidence,city,policies_violated_gold,review,tokenized-review,compound,neg,neu,pos
0,679455653,False,finalized,3,2/21/15 0:36,"[RudeService, OrderProblem, Filthy]","[1.0, 0.6667, 0.6667]",Atlanta,NaN,im huge mcds lover ive better ones this far wo...,"[im, huge, mcds, lover, ive, better, ones, thi...",0.0790,0.253,0.488,0.259
1,679455654,False,finalized,3,2/21/15 0:27,[RudeService],[1.0],Atlanta,NaN,terrible customer service i came 930pm stood f...,"[terrible, customer, service, i, came, 930pm, ...",-0.7430,0.192,0.725,0.083
2,679455655,False,finalized,3,2/21/15 0:26,"[SlowService, OrderProblem]","[1.0, 1.0]",Atlanta,NaN,first lost order actually gave someone one els...,"[first, lost, order, actually, gave, someone, ...",-0.7351,0.132,0.815,0.053
3,679455656,False,finalized,3,2/21/15 0:27,[na],[0.6667],Atlanta,NaN,i see im one giving 1 star only 25 star thats ...,"[i, see, im, one, giving, 1, star, only, 25, s...",0.3400,0.000,0.844,0.156
4,679455657,False,finalized,3,2/21/15 0:27,[RudeService],[1.0],Atlanta,NaN,well mcdonalds know food is this review reflec...,"[well, mcdonalds, know, food, is, this, review...",0.8934,0.106,0.665,0.229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520,679500008,False,finalized,3,2/21/15 0:13,[OrderProblem],[0.6754],Portland,NaN,i enjoyed part i repeatedly asked i right sauc...,"[i, enjoyed, part, i, repeatedly, asked, i, ri...",0.7311,0.000,0.661,0.339
1521,679500224,False,finalized,3,2/21/15 0:27,"[SlowService, Filthy, BadFood]","[1.0, 1.0, 1.0]",Houston,NaN,worst mcdonalds ive long time dirt everywhere ...,"[worst, mcdonalds, ive, long, time, dirt, ever...",-0.7579,0.351,0.649,0.000
1522,679500608,False,finalized,3,2/21/15 0:24,[ScaryMcDs],[0.6458],New York,NaN,when i really craving mcdonalds seems closest ...,"[when, i, really, craving, mcdonalds, seems, c...",0.8885,0.000,0.747,0.253
1523,679501257,False,finalized,3,2/21/15 0:32,[ScaryMcDs],[0.6407],Chicago,NaN,two points right gate 1 thuggery knows race li...,"[two, points, right, gate, 1, thuggery, knows,...",-0.8627,0.109,0.819,0.072


In [68]:
# categorize reviews as expected negatieve or positive

csv['evaluation'] = csv[['neg','pos']].idxmax(axis=1)

print(csv['evaluation'].value_counts())
csv.head(5)

pos    768
neg    757
Name: evaluation, dtype: int64


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,policies_violated,policies_violated:confidence,city,policies_violated_gold,review,tokenized-review,compound,neg,neu,pos,evaluation
0,679455653,False,finalized,3,2/21/15 0:36,"[RudeService, OrderProblem, Filthy]","[1.0, 0.6667, 0.6667]",Atlanta,NaN,im huge mcds lover ive better ones this far wo...,"[im, huge, mcds, lover, ive, better, ones, thi...",0.0790,0.253,0.488,0.259,pos
1,679455654,False,finalized,3,2/21/15 0:27,[RudeService],[1.0],Atlanta,NaN,terrible customer service i came 930pm stood f...,"[terrible, customer, service, i, came, 930pm, ...",-0.7430,0.192,0.725,0.083,neg
2,679455655,False,finalized,3,2/21/15 0:26,"[SlowService, OrderProblem]","[1.0, 1.0]",Atlanta,NaN,first lost order actually gave someone one els...,"[first, lost, order, actually, gave, someone, ...",-0.7351,0.132,0.815,0.053,neg
3,679455656,False,finalized,3,2/21/15 0:27,[na],[0.6667],Atlanta,NaN,i see im one giving 1 star only 25 star thats ...,"[i, see, im, one, giving, 1, star, only, 25, s...",0.3400,0.000,0.844,0.156,pos
4,679455657,False,finalized,3,2/21/15 0:27,[RudeService],[1.0],Atlanta,NaN,well mcdonalds know food is this review reflec...,"[well, mcdonalds, know, food, is, this, review...",0.8934,0.106,0.665,0.229,pos


In [80]:
# This data set does not contain golden reviews, hence we can drop these columns as we are not interested.

print(csv[['_golden']].value_counts())

# Dropping columns that are not usefull and create new dataframe reviews

reviews = csv.drop(columns=['_unit_id', '_golden', '_last_judgment_at', 'policies_violated_gold', '_unit_state'], axis=1)

reviews.head(5)

_golden
False      1525
dtype: int64


,_trusted_judgments,policies_violated,policies_violated:confidence,city,review,tokenized-review,compound,neg,neu,pos,evaluation
0,3,"[RudeService, OrderProblem, Filthy]","[1.0, 0.6667, 0.6667]",Atlanta,im huge mcds lover ive better ones this far wo...,"[im, huge, mcds, lover, ive, better, ones, thi...",0.0790,0.253,0.488,0.259,pos
1,3,[RudeService],[1.0],Atlanta,terrible customer service i came 930pm stood f...,"[terrible, customer, service, i, came, 930pm, ...",-0.7430,0.192,0.725,0.083,neg
2,3,"[SlowService, OrderProblem]","[1.0, 1.0]",Atlanta,first lost order actually gave someone one els...,"[first, lost, order, actually, gave, someone, ...",-0.7351,0.132,0.815,0.053,neg
3,3,[na],[0.6667],Atlanta,i see im one giving 1 star only 25 star thats ...,"[i, see, im, one, giving, 1, star, only, 25, s...",0.3400,0.000,0.844,0.156,pos
4,3,[RudeService],[1.0],Atlanta,well mcdonalds know food is this review reflec...,"[well, mcdonalds, know, food, is, this, review...",0.8934,0.106,0.665,0.229,pos


In [69]:
# TODO : fix the rest